# Using Ego Graphs to Visualize recommendations

This notebook uses reclibwh's `RecommenderEnsemble` class to generate pairwise item-item similarities. This gives a ranking ordering of each item w.r.t every other item.

The generated pairwise distance matrix is dense, and if a weighted graph is generated from it, would make it almost impossible to visualize unless you're a computer (but computers don't watch movies!).

In order to make this visualization more legible, some sparsity has to be enforced, by aggresively pruning the graph. A method is to use an item as a target to visualization (think "what movies are most similar to The Big Lebowski") and take the top K items (in this demo K=200). This still leaves K^2 = 40000 in this case. 

A next step would then be to only draw connections for the top M matches for each movie, use the inverse ranking as the weight of each edge and take an ego graph about the target item, and limit the graph to a distance threshold (D=10 in this example).

K, M, and D are all tunable parameters of the visualization, and can be left up to the user to customize!

Credits to https://medium.com/applied-data-science/the-google-vs-trick-618c8fd5359f from which I borrowed the ego graph pruning idea

In [ ]:
import os, sys
os.sys.path.append('..')
from reclibwh.utils.ItemMetadata import ExplicitDataFromCSV
from reclibwh.recommender.recommenderMF import RecommenderMFAsymCached
from reclibwh.recommender.recommenderALS import RecommenderALS
from reclibwh.recommender.recommenderEnsemble import RecommenderEnsemble
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from tqdm import tqdm

# PLOTLY
import plotly
import plotly.offline as py
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

## User defined variables

In [64]:
DATA_FOLDER = '~/personal/recommender/data/ml-20m-2'
MODEL_FOLDER = '/home/ong/personal/recommender/models'
OUTPUT_FOLDER = '/home/ong/personal/recommender/notebooks'
IMPLICIT_MODEL = '2020_06_04_ALS_latest_trained'
EXPLICIT_MODEL = '2020_06_04_MF_latest_trained/ASVDC'

## Ego graph generation variables

In [69]:
K = 200  # Number of closest neighbours to evaluate on target
DT = 10 # Ego graph distance threshold
M = 5 # Top suggestions per movie to consider

## Load data and models

In [3]:
exp_data = ExplicitDataFromCSV(True, data_folder=DATA_FOLDER)
model_path = os.path.join(MODEL_FOLDER, IMPLICIT_MODEL)
rals = RecommenderALS(mode='predict', model_path=model_path)
model_path = os.path.join(MODEL_FOLDER, EXPLICIT_MODEL)
rmfa = RecommenderMFAsymCached(mode='predict', model_path=model_path) 
rec = RecommenderEnsemble([rals, rmfa])
rec.input_data(exp_data)

/home/ong/personal/recommender/models/2020_06_04_MF_latest_trained/ASVDC SVD_asym_cached.json.template
Model: "model_8"
________________________________________________________________________________________
Layer (type)                 Output Shape       Param #   Connected to                  
ruj_in (InputLayer)          [(None, None)]     0                                       
________________________________________________________________________________________
bj_in (InputLayer)           [(None, None)]     0                                       
________________________________________________________________________________________
ui_in (InputLayer)           [(None, None)]     0                                       
________________________________________________________________________________________
roffset (Subtract)           (None, None)       0         ruj_in[0][0]                  
                                                          bj_in[0][0]          

## Retrieve predictions and features for all movies

In [4]:
F = rec.predict(np.zeros(shape=(exp_data.M), dtype=int),np.arange(exp_data.M))

In [5]:
features = np.stack([F[0]['q'], F[1]['p']]).transpose([1,0,2])

In [6]:
features.shape

(26744, 2, 20)

In [7]:
D = exp_data.stats.merge(exp_data.md_df, left_index=True, right_on='item_cat', how='inner')
D = D[['title', 'user']].merge(pd.DataFrame({'features': [f for f in features]}), left_index=True, right_index=True)
D.head()

,title,user,features
item_cat,,,
0,Toy Story (1995),49695,"[[0.814873944562587, -0.9939122556224087, 0.85..."
1,Jumanji (1995),22243,"[[0.6984649529435087, -1.0635424225910692, 0.7..."
2,Grumpier Old Men (1995),12735,"[[0.5243329238314206, -0.9487003977463584, 0.2..."
3,Waiting to Exhale (1995),2756,"[[0.7911500328247573, -0.5434130290354404, -0...."
4,Father of the Bride Part II (1995),12161,"[[0.5367890956671346, -0.916618998396046, 0.36..."


## Extract K most similar movies to a target movie, to narrow down the visual range

In [71]:
kw = 'Big Lebowski'
query_ids = np.array([f for f in D.loc[D.title.str.startswith(kw)].index])
source = query_ids[0]
D.loc[D.title.str.startswith(kw), ['title']]

,title
item_cat,
1672,"Big Lebowski, The (1998)"


In [72]:
items, dists = rec.similar_to([source])
rec_items = items[0]
Drec = D.loc[rec_items, ['user', 'title', 'features']].iloc[0:K]
Drec.head()

,user,title,features
item_cat,,,
1672,20725,"Big Lebowski, The (1998)","[[0.5391080817355389, -0.9697238414590742, 0.9..."
4883,12513,"Royal Tenenbaums, The (2001)","[[0.8013755752829664, -0.666830065126893, 1.04..."
9583,5424,"Life Aquatic with Steve Zissou, The (2004)","[[0.7451982039609414, -0.9841446440214618, 1.1..."
3933,19459,"O Brother, Where Art Thou? (2000)","[[0.7590744845174403, -0.8168896562412145, 1.0..."
2417,20328,Office Space (1999),"[[0.6468789998820698, -0.7741868833864933, 1.1..."


In [73]:
Qrec = np.array([f for f in Drec['features']])
Qrec = Qrec.transpose([1,0,2])
similarity = np.mean(np.stack([cosine_similarity(q) for q in Qrec]), axis=0)

In [74]:
ranked = np.argsort(similarity, axis=1)[:,::-1]
ranked = ranked[:, 1:1+M]
ranked

array([[  1,   3,   2,   4,   5],
       [  5,   2,   0,   9,  10],
       [  9,   1,  16,   0,  15],
       [  0,   1,  31,  26,  21],
       [  0,  28,  38, 166, 101],
       [  1,   8,   0,  53,  14],
       [ 72, 147,   0,   2,  11],
       [ 32,  21,  14,  46,  69],
       [ 42, 124,  15,  53,   5],
       [  2,   1, 159,  68, 126],
       [ 16,  13,  53,  58, 156],
       [ 31,   3,  23, 197, 101],
       [ 13,  10, 156,  60, 113],
       [ 10,  12,  39,  60,  47],
       [  7,  55,   5, 196,  71],
       [ 58,  10,  16,  60,   8],
       [ 10,  15,   2,  47,  60],
       [ 23,  24,  45,  37, 184],
       [ 64,  77,  88, 146,  98],
       [ 12, 107,  42,  10,  21],
       [ 48,  23, 111,  17,  37],
       [  7,  12, 164,   3, 149],
       [ 37,  68,  12, 156,  69],
       [ 20,  24,  17,  37,  48],
       [ 23, 110,  17,  98,  32],
       [196,  54, 129,  55,  94],
       [123,  74, 100,   3,  61],
       [ 79,  97, 127, 135,  54],
       [ 65, 190, 195,   4, 143],
       [ 41,  

## Make a weighted ego graph centered around query item

In [75]:
def compute_weights(R):
    weights = {}
    dist = {}
    N = R.shape[1]
    for r in range(len(R)):
        P = R[r]
        for j, p in enumerate(P):
            e = tuple(sorted((r,p)))
            if e in weights: weights[e] += N-j
            else: weights[e] = N-j
    for e, w in weights.items():
        dist[e] = 2*N - w
        
    return weights, dist

In [76]:
weights, dist = compute_weights(ranked)

In [77]:
nodes = [(i, {'title': Drec.iloc[i]['title']}) for i in range(len(ranked))]
edges = [(e[0], e[1], {'distance': d}) for e, d in dist.items()]
G=nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [78]:
print("computing ego graph")
center = 0
EG = nx.ego_graph(G, center, distance = 'distance', radius = DT)

print("computing k-connected subgraphs")
#FIND THE 2-CONNECTED SUBGRAPHS
subgraphs = nx.algorithms.connectivity.edge_kcomponents.k_edge_subgraphs(EG, k = 3)

print("searching through subgraphs")
#GET THE SUBGRAPH THAT CONTAINS center
for s in subgraphs:
    if center in s:
        print(s)
        break
pruned_EG = EG.subgraph(s)

computing ego graph
computing k-connected subgraphs
searching through subgraphs
{0, 1, 2, 3, 4, 5, 6, 128, 136, 9, 10, 11, 142, 143, 147, 28, 157, 30, 31, 159, 162, 38, 166, 168, 44, 50, 53, 189, 62, 65, 66, 195, 68, 197, 72, 75, 95, 117}


## Visualize Ego graph

In [79]:
# Visualization function from plotly
def visualize_graph(G, key=None):
    
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            )))

    node_adjacencies = []
    node_text = [G.nodes[node]['title'] for node in G.nodes]
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        # node_text.append('# of connections: '+str(len(adjacencies[1])))

    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text

    return edge_trace, node_trace

In [80]:
pos = nx.spring_layout(pruned_EG)
nx.set_node_attributes(pruned_EG, pos, 'pos')
node_trace, edge_trace = visualize_graph(pruned_EG)

In [81]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
py.offline.iplot(fig)

## Export graph for visualization elsewhere

In [82]:
nodes = Drec[['title']].copy()
nodes['idx'] = np.arange(len(nodes))
node_degrees = pd.DataFrame({
    'idx': [n[0] for n in pruned_EG.degree],
    'degree': [n[1] for n in pruned_EG.degree]
})
nodes = nodes.merge(node_degrees, on='idx')
nodes.set_index('idx', inplace=True)

edges = [e for e in pruned_EG.edges.data('distance')]
edges = pd.DataFrame({
    'u': [nodes.loc[e[0], 'title'] for e in edges],
    'v': [nodes.loc[e[1], 'title'] for e in edges],
    'd': [e[2] for e in edges],
})
edges.to_csv(os.path.join(OUTPUT_FOLDER, 'edges.csv'), index=False)
nodes.to_csv(os.path.join(OUTPUT_FOLDER, 'nodes.csv'), index=False)

nodes


,title,degree
idx,,
0,"Big Lebowski, The (1998)",8
1,"Royal Tenenbaums, The (2001)",8
2,"Life Aquatic with Steve Zissou, The (2004)",6
3,"O Brother, Where Art Thou? (2000)",4
4,Office Space (1999),7
5,Rushmore (1998),3
6,Fear and Loathing in Las Vegas (1998),5
9,"Darjeeling Limited, The (2007)",5
10,Adaptation (2002),5
